In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [33]:
# Reads data
features = pd.read_csv("data/features.csv")
target = pd.read_csv("data/target.csv")

In [34]:
# Bins target values
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=4, encode = "onehot-dense", strategy = "quantile")

target_discretized = discretizer.fit_transform(target["G3"].values.reshape(-1, 1))
target_new = []
for row in target_discretized:
    target_new.append(list(row).index(1))

In [35]:
# Splits data into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 1001)

In [36]:
# Gets final grade for target
t_train = y_train["G3"]
t_test = y_test["G3"]

In [37]:
# Imports Ridge and accuracy metrics
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs Ridge Regression
model = Ridge().fit(X=X_train, y=t_train)

# Measures accuracy
print("R-squared value for training set: ", r2_score(t_train, model.predict(X_train)))
print("R-squared value for testing set: ", r2_score(t_test, model.predict(X_test)))

R-squared value for training set:  0.34042660318538653
R-squared value for testing set:  0.2277177794229266


In [38]:
# Splits data for classification into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_new, random_state = 1001)

In [39]:
# Imports SVC and accuracy metrics
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Recommended using kernel

# Performs SVC
model = SVC(gamma = "scale").fit(X=X_train, y=y_train)

# Measures accuracy
accuracy_train = model.score(X_train, y_train)
accuracy_test = model.score(X_test, y_test)
print("Prediction accuracy on the train data:", f"{accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{accuracy_test:.2%}")

Prediction accuracy on the train data: 37.42%
Prediction accuracy on the test data: 33.72%


In [40]:
# Imports Decision Tree and accuracy metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs Decision Tree Classifier
model = DecisionTreeClassifier().fit(X=X_train, y=y_train)

# Recommended using kernel -> we want to be able to model nonlinear things!
# Want to use feature expansion to classify them
# To choose the kernel: brute force/ try out a bunch of different ones (look at data and see if there's anything interesting you can pick up on, for example see if there's any 2 or more things that you can take the product of and it seems interesting)
# kernel: considering n features at the same time

# Measures accuracy
accuracy_train = model.score(X_train, y_train)
accuracy_test = model.score(X_test, y_test)
print("Prediction accuracy on the train data:", f"{accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{accuracy_test:.2%}")

Prediction accuracy on the train data: 100.00%
Prediction accuracy on the test data: 38.31%


In [69]:
import torch

class NeuralNetwork(torch.nn.Module):
    def __init__(self, inputDim, outputDim, layerDim):
        
        super(NeuralNetwork, self).__init__()
        self.inputDim = inputDim
        self.outputDim = outputDim
        self.layerDim = layerDim


        self.l1 = torch.nn.Linear(self.inputDim, self.layerDim)
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(self.layerDim, self.outputDim)
        self.relu = torch.nn.ReLU()

        
    def forward(self, x):
        hidden = self.l1(x)
        relu = self.relu(hidden)
        output = self.l2(relu)
        # output = self.relu(output)
        return output

    def score(self, x_test, y_test):
        x_test_tensor = torch.FloatTensor(X_test.values)
        test_results = model.forward(x_test_tensor)
        results = [torch.argmax(res).item() for res in test_results]
        
        total = 0
        for i in range(len(results)):
            if results[i] == y_test[i]:
                total += 1

        return total/len(y_test)


In [70]:
model = NeuralNetwork(96, 4, 48)


In [72]:
import numpy as np
# Now have to loop through all my data points to optimize my model - for loop looping through groups of (5) data points at a time using indexing

criterion = torch.nn.CrossEntropyLoss()
#loss = criterion(y_pred, torch.tensor(np.array(y_train).reshape(-1)))

# model.eval()
# x_test_tensor = torch.FloatTensor(X_test.iloc[3].values[1:])
# y_pred = model(x_test_tensor)
# before_train = criterion(y_pred.squeeze(), torch.FloatTensor(t_test.values[0]))
# print('Test loss before training' , before_train.item())

# training mode vs. evaluation mode
# different architecture @ training/eval times
# model.train()
# y_pred = model(torch.FloatTensor(X_train.iloc[3].values[1:]))
# y_pred

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# 2 inputs, first is model.parameters() - includes all parameters in the model you want to optimize, make sure model.parameters() includes everything you want to optimize

for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    x_train_tensor = torch.FloatTensor(X_train.values)
    y_pred = model(x_train_tensor)

    # Compute and print loss.
    loss = criterion(y_pred, torch.tensor(pd.DataFrame(y_train).values).reshape(-1))
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

99 1.5904982089996338
199 1.353495717048645
299 1.2996772527694702
399 1.275328516960144
499 1.2552828788757324


In [73]:
x_test_tensor = torch.FloatTensor(X_test.values)
test_results = model.forward(x_test_tensor)

In [75]:
results = [torch.argmax(res).item() for res in test_results]
model.score(X_train, y_train)

0.10600255427841634

In [49]:
model.forward(x_test_tensor)[1]

tensor([ -1.0928,  -3.3668,  -8.8386, -17.7452, -23.2715,  -3.1942,  -2.3790,
         -1.6477,  -0.6047,  -0.6456,   0.1171,  -0.0592,   0.0371,  -0.0275,
         -0.5069,  -0.4071,  -0.8462,  -1.0783,  -1.3931,  -3.1473],
       grad_fn=<SelectBackward>)

In [50]:
t_test

848     8
529    12
348    15
805    17
45      6
       ..
356    13
936    11
724    14
962     0
779    10
Name: G3, Length: 261, dtype: int64